# Alura - 7DaysOfCode - Python Pandas

The objective of second day challenge is to begin preparing the loan data from UFRN libraries' collections over the past 10 years. To achieve this we will removing unnecessary data, grouping data, assigning new information.d

- <i>Day 1: Importing the data</i>
- <i><b>Day 2: Data Preparation & Context</b></i>
    1. Create a new column for UDC class based on the location
    2. Remove unnecessary data, delete "registro_sistema"
    3. Modify data format, convert the "matricula_ou_siape" column to string 

## Day 1: Importing the data

### 1. Import loan data from the last 10 years.

In [43]:
import os
import pandas as pd

In [44]:
fpaths = os.listdir('../datasets/dados_emprestimos')
loans = pd.DataFrame()

for fpath in fpaths:
    tmp = pd.read_csv(f'../datasets/dados_emprestimos/{fpath}')
    loans = pd.concat([loans, tmp])

In [45]:
print("Numbers of loans: ", loans.shape[0])
print("Numbers of columns: ", loans.shape[1])
loans.head(5)

Numbers of loans:  2258018
Numbers of columns:  7


,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,9880640,2015035868,NaN,2017/07/03 07:11:59.514000000,2017/07/03 07:12:27.000000000,2.016043e+09,ALUNO DE GRADUAÇÃO
1,9880644,Y012079,2017/07/18 17:56:06.584000000,2017/07/03 07:50:51.650000000,2017/08/09 18:53:58.000000000,2.015013e+10,ALUNO DE GRADUAÇÃO
2,9880648,2014086285,2017/08/02 03:03:43.220000000,2017/07/03 08:20:56.988000000,2017/09/01 07:48:33.000000000,2.017102e+10,ALUNO DE PÓS-GRADUAÇÃO
3,9880652,2014083935,2017/08/02 03:03:43.197000000,2017/07/03 08:20:57.085000000,2017/09/01 07:48:51.000000000,2.017102e+10,ALUNO DE PÓS-GRADUAÇÃO
4,9880656,2014086281,2017/08/02 03:03:43.175000000,2017/07/03 08:20:57.144000000,2017/09/01 07:49:01.000000000,2.017102e+10,ALUNO DE PÓS-GRADUAÇÃO


###  2. Import inventory item data.

In [46]:
fpath = '../datasets/dados_exemplares.parquet'
books = pd.read_parquet(fpath)

In [47]:
print("Number of books: ", books.shape[0])
print("Number of columns: ", books.shape[1])
books.head(5)

Number of books:  546237
Number of columns:  7


,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


### 3. Merge loan and inventory item data into a single DataFrame using the barcode

In [48]:
loans_books = pd.merge(left=loans, right=books, on='codigo_barras')

In [49]:
loans_books.head(3)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,9880640,2015035868,NaN,2017/07/03 07:11:59.514000000,2017/07/03 07:12:27.000000000,2.016043e+09,ALUNO DE GRADUAÇÃO,1217020,Acervo Circulante,Biblioteca Setorial Profª. Maria Lúcia da Cost...,REGULAR,208,214215
1,9880648,2014086285,2017/08/02 03:03:43.220000000,2017/07/03 08:20:56.988000000,2017/09/01 07:48:33.000000000,2.017102e+10,ALUNO DE PÓS-GRADUAÇÃO,1173407,Acervo Circulante,Biblioteca Setorial Prof. Leopoldo Nelson - ­C...,REGULAR,151,214418
2,9880652,2014083935,2017/08/02 03:03:43.197000000,2017/07/03 08:20:57.085000000,2017/09/01 07:48:51.000000000,2.017102e+10,ALUNO DE PÓS-GRADUAÇÃO,1173614,Acervo Circulante,Biblioteca Setorial Prof. Leopoldo Nelson - ­C...,REGULAR,141,213367


### 4. Clean the data by handling null and duplicate values.

In [50]:
print('Numebr of null values per column:')
loans_books.isnull().sum()

Numebr of null values per column:


id_emprestimo                 0
codigo_barras                 0
data_renovacao          1162088
data_emprestimo               0
data_devolucao             6178
matricula_ou_siape         2893
tipo_vinculo_usuario          0
id_exemplar                   0
colecao                       0
biblioteca                    0
status_material               0
localizacao                   0
registro_sistema              0
dtype: int64

In [51]:
# Drop all rows that have a loan without student 
loans_books.dropna(subset=['matricula_ou_siape'], axis=0, inplace=True)

In [52]:
print('Number of duplicated rows:', loans_books.duplicated().sum())

Number of duplicated rows: 36


In [53]:
# Drop all duplicates rows
loans_books.drop_duplicates(inplace=True)

## Day 2: Data preparation & Context

### 1. Create a new column for UDC class based on the location
UDC main class:
- 000-099: Generalities. Science and knowledge.
- 100-199: Philosophy and psychology.
- 200-299: Religion.
- 300-399: Social sciences.
- 400-499: Unassigned class. Temporarily not in use.
- 500-599: Mathematics and natural sciences.
- 600-699: Technology (Applied sciences).
- 700-799: Arts.
- 800-899: Literature. Rhetoric. Literary criticism.
- 900-999: Geography. Biography. History.

In [54]:
udc = {
    0: "Generalities. Science and knowledge.",
    1: "Philosophy and psychology.",
    2: "Religion.",
    3: "Social sciences.",
    4: "Unassigned class. Temporarily not in use.",
    5: "Mathematics and natural sciences.",
    6: "Technology (Applied sciences).",
    7: "Arts.",
    8: "Literature. Rhetoric. Literary criticism.",
    9: "Geography. Biography. History."
}
def code2udc(code: int):
    return udc[code//100] if code//100 in udc else -1

In [55]:
loans_books['udc'] = loans_books['localizacao'].apply(lambda loc: code2udc(int(loc)))

In [56]:
loans_books[['id_exemplar', 'udc']].head(3)

,id_exemplar,udc
0,1217020,Religion.
1,1173407,Philosophy and psychology.
2,1173614,Philosophy and psychology.


### 2. Remove unnecessary data, delete "registro_sistema"

In [57]:
loans_books.drop('registro_sistema', axis=1, inplace = True)

### 3. Modify data format, convert the "matricula_ou_siape" column to string 

In [60]:
# First format as int, to exclud the point float from the final string
loans_books['matricula_ou_siape'] = loans_books['matricula_ou_siape'].astype('int').astype('str')

In [59]:
loans_books[['id_emprestimo', 'matricula_ou_siape']].head(3)

,id_emprestimo,matricula_ou_siape
0,9880640,2016042604
1,9880648,20171015563
2,9880652,20171015563
